# Obtaining metadata for countries of Spotify charts dataset

To make the information about the countries (regions that are not `"Global"`) in the chart data even more insightful, we should get the country ISO codes as well. Furthermore, region information could be useful.

In [1]:
import pandas as pd
from helpers import get_data_path, create_data_out_path

## Loading charts data

In [2]:
charts = pd.read_csv(get_data_path("top50.csv"))
charts

,region,date,rank,streams,id
0,Argentina,2017-01-01,1,253019.0,6mICuAdrwEjh6Y6lroV2Kg
1,Argentina,2017-01-01,2,223988.0,7DM4BPaS7uofFul3ywMe46
2,Argentina,2017-01-01,3,210943.0,3AEZUABDXNtecAOSC1qTfo
3,Argentina,2017-01-01,4,173865.0,6rQSrBHf7HlZjtcMZ4S4bO
4,Argentina,2017-01-01,5,153956.0,58IL315gMSTD37DOZPJ2hf
...,...,...,...,...,...
4563811,Uruguay,2021-08-13,25,11431.0,4aXicRG47bBEcKXlDaSRwW
4563812,Uruguay,2021-08-13,27,10946.0,43qcs9NpJhDxtG91zxFkj7
4563813,Uruguay,2021-08-13,33,9994.0,4M4DTyLupKvbJKlE4I4EYq
4563814,Uruguay,2021-08-13,35,9251.0,5jzEwSyyymBlf1fa1o39T2


## Loading ISO code data

In [3]:
country_iso_data = pd.read_csv(
    get_data_path(
        "iso_codes.csv",
        download_url="https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv",
    ),
).set_index("alpha-3")
country_iso_data

,name,alpha-2,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
alpha-3,,,,,,,,,,
AFG,Afghanistan,AF,4,ISO 3166-2:AF,Asia,Southern Asia,NaN,142.0,34.0,NaN
ALA,Åland Islands,AX,248,ISO 3166-2:AX,Europe,Northern Europe,NaN,150.0,154.0,NaN
ALB,Albania,AL,8,ISO 3166-2:AL,Europe,Southern Europe,NaN,150.0,39.0,NaN
DZA,Algeria,DZ,12,ISO 3166-2:DZ,Africa,Northern Africa,NaN,2.0,15.0,NaN
ASM,American Samoa,AS,16,ISO 3166-2:AS,Oceania,Polynesia,NaN,9.0,61.0,NaN
...,...,...,...,...,...,...,...,...,...,...
WLF,Wallis and Futuna,WF,876,ISO 3166-2:WF,Oceania,Polynesia,NaN,9.0,61.0,NaN
ESH,Western Sahara,EH,732,ISO 3166-2:EH,Africa,Northern Africa,NaN,2.0,15.0,NaN
YEM,Yemen,YE,887,ISO 3166-2:YE,Asia,Western Asia,NaN,142.0,145.0,NaN


## Joining Spotify regions with ISO code dataframe

In [4]:
countries = list(set(charts.reset_index().region))
countries.remove("Global")
len(countries)

49

In [5]:
matches = (
    country_iso_data[country_iso_data["name"].isin(countries)][["name"]]
    .copy()
    .rename(columns={"name": "spotify_region"})
)
mismatches = list(set(countries) - set(matches.spotify_region.to_list()))


In [6]:
matches

,spotify_region
alpha-3,
ARG,Argentina
AUS,Australia
AUT,Austria
BEL,Belgium
BRA,Brazil
CAN,Canada
CHL,Chile
COL,Colombia
CRI,Costa Rica


In [7]:
mismatches

['Czech Republic', 'United Kingdom', 'United States', 'Bolivia', 'Taiwan']

In [8]:
def search_for_country_name_substr(substr):
  return country_iso_data[country_iso_data.name.str.lower().str.contains(substr.lower())]

In [9]:
search_for_country_name_substr("Bol")

,name,alpha-2,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
alpha-3,,,,,,,,,,
BOL,Bolivia (Plurinational State of),BO,68,ISO 3166-2:BO,Americas,Latin America and the Caribbean,South America,19.0,419.0,5.0
VEN,Venezuela (Bolivarian Republic of),VE,862,ISO 3166-2:VE,Americas,Latin America and the Caribbean,South America,19.0,419.0,5.0


In [10]:
manual_mappings = []

In [11]:
manual_mappings.append(("Bolivia", "BOL"))

In [12]:
search_for_country_name_substr("Cze")

,name,alpha-2,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
alpha-3,,,,,,,,,,
CZE,Czechia,CZ,203,ISO 3166-2:CZ,Europe,Eastern Europe,NaN,150.0,151.0,NaN


In [13]:
manual_mappings.append(("Czech Republic", "CZE"))

In [14]:
search_for_country_name_substr("Taiwan")

,name,alpha-2,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
alpha-3,,,,,,,,,,
TWN,"Taiwan, Province of China",TW,158,ISO 3166-2:TW,Asia,Eastern Asia,NaN,142.0,30.0,NaN


In [15]:
manual_mappings.append(("Taiwan", "TWN"))

In [16]:
search_for_country_name_substr("United Kingdom")

,name,alpha-2,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
alpha-3,,,,,,,,,,
GBR,United Kingdom of Great Britain and Northern I...,GB,826,ISO 3166-2:GB,Europe,Northern Europe,NaN,150.0,154.0,NaN


In [17]:
manual_mappings.append(("United Kingdom", "GBR"))

In [18]:
search_for_country_name_substr("United States")

,name,alpha-2,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
alpha-3,,,,,,,,,,
USA,United States of America,US,840,ISO 3166-2:US,Americas,Northern America,NaN,19.0,21.0,NaN
UMI,United States Minor Outlying Islands,UM,581,ISO 3166-2:UM,Oceania,Micronesia,NaN,9.0,57.0,NaN


In [19]:
manual_mappings.append(("United States", "USA"))

In [20]:
manual_mapping_df = pd.DataFrame(manual_mappings, columns=["spotify_region", "alpha-3"]).set_index("alpha-3")

In [21]:
spotify_country_iso_alpha3 = pd.concat([matches, manual_mapping_df])

In [22]:
spotify_country_iso_alpha3

,spotify_region
alpha-3,
ARG,Argentina
AUS,Australia
AUT,Austria
BEL,Belgium
BRA,Brazil
CAN,Canada
CHL,Chile
COL,Colombia
CRI,Costa Rica


In [23]:
spotify_country_iso_alpha3.shape

(49, 1)

In [24]:
spotify_country_iso_alpha3.columns

Index(['spotify_region'], dtype='object')

In [25]:
country_iso_data.columns

Index(['name', 'alpha-2', 'country-code', 'iso_3166-2', 'region', 'sub-region',
       'intermediate-region', 'region-code', 'sub-region-code',
       'intermediate-region-code'],
      dtype='object')

In [26]:
country_iso_data

,name,alpha-2,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
alpha-3,,,,,,,,,,
AFG,Afghanistan,AF,4,ISO 3166-2:AF,Asia,Southern Asia,NaN,142.0,34.0,NaN
ALA,Åland Islands,AX,248,ISO 3166-2:AX,Europe,Northern Europe,NaN,150.0,154.0,NaN
ALB,Albania,AL,8,ISO 3166-2:AL,Europe,Southern Europe,NaN,150.0,39.0,NaN
DZA,Algeria,DZ,12,ISO 3166-2:DZ,Africa,Northern Africa,NaN,2.0,15.0,NaN
ASM,American Samoa,AS,16,ISO 3166-2:AS,Oceania,Polynesia,NaN,9.0,61.0,NaN
...,...,...,...,...,...,...,...,...,...,...
WLF,Wallis and Futuna,WF,876,ISO 3166-2:WF,Oceania,Polynesia,NaN,9.0,61.0,NaN
ESH,Western Sahara,EH,732,ISO 3166-2:EH,Africa,Northern Africa,NaN,2.0,15.0,NaN
YEM,Yemen,YE,887,ISO 3166-2:YE,Asia,Western Asia,NaN,142.0,145.0,NaN


In [27]:
spotify_country_iso_data = spotify_country_iso_alpha3.join(country_iso_data)
spotify_country_iso_data

,spotify_region,name,alpha-2,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
alpha-3,,,,,,,,,,,
ARG,Argentina,Argentina,AR,32,ISO 3166-2:AR,Americas,Latin America and the Caribbean,South America,19.0,419.0,5.0
AUS,Australia,Australia,AU,36,ISO 3166-2:AU,Oceania,Australia and New Zealand,NaN,9.0,53.0,NaN
AUT,Austria,Austria,AT,40,ISO 3166-2:AT,Europe,Western Europe,NaN,150.0,155.0,NaN
BEL,Belgium,Belgium,BE,56,ISO 3166-2:BE,Europe,Western Europe,NaN,150.0,155.0,NaN
BRA,Brazil,Brazil,BR,76,ISO 3166-2:BR,Americas,Latin America and the Caribbean,South America,19.0,419.0,5.0
CAN,Canada,Canada,CA,124,ISO 3166-2:CA,Americas,Northern America,NaN,19.0,21.0,NaN
CHL,Chile,Chile,CL,152,ISO 3166-2:CL,Americas,Latin America and the Caribbean,South America,19.0,419.0,5.0
COL,Colombia,Colombia,CO,170,ISO 3166-2:CO,Americas,Latin America and the Caribbean,South America,19.0,419.0,5.0
CRI,Costa Rica,Costa Rica,CR,188,ISO 3166-2:CR,Americas,Latin America and the Caribbean,Central America,19.0,419.0,13.0


Wow, that was painful.

In [28]:
spotify_country_iso_data.columns

Index(['spotify_region', 'name', 'alpha-2', 'country-code', 'iso_3166-2',
       'region', 'sub-region', 'intermediate-region', 'region-code',
       'sub-region-code', 'intermediate-region-code'],
      dtype='object')

In [29]:
spotify_country_iso_data = spotify_country_iso_data.reset_index()[
    ["spotify_region", "alpha-3", "alpha-2", "region", "sub-region"]
].rename(
    columns={
        "alpha-3": "iso_alpha3",
        "alpha-2": "iso_alpha2",
        "region": "geo_region",
        "sub-region": "geo_subregion",
    }
)
spotify_country_iso_data


,spotify_region,iso_alpha3,iso_alpha2,geo_region,geo_subregion
0,Argentina,ARG,AR,Americas,Latin America and the Caribbean
1,Australia,AUS,AU,Oceania,Australia and New Zealand
2,Austria,AUT,AT,Europe,Western Europe
3,Belgium,BEL,BE,Europe,Western Europe
4,Brazil,BRA,BR,Americas,Latin America and the Caribbean
5,Canada,CAN,CA,Americas,Northern America
6,Chile,CHL,CL,Americas,Latin America and the Caribbean
7,Colombia,COL,CO,Americas,Latin America and the Caribbean
8,Costa Rica,CRI,CR,Americas,Latin America and the Caribbean
9,Denmark,DNK,DK,Europe,Northern Europe


## Export data

In [30]:
spotify_country_iso_data.to_csv(create_data_out_path("spotify_region_metadata.csv"), index=False)

Data output path: /home/sejmou/Repos/Uni/VisDS/vis-ds/data/spotify_region_metadata.csv
